In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from string import punctuation
import csv
import pandas

np.random.seed(1)

Using TensorFlow backend.


In [2]:
df = pandas.read_csv("data/train.csv", index_col="id")
df.head()

D:\Tools\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
id,,,,,,,,,,,,,,,,,,,,,
59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [3]:
nparray = df.values
X = nparray[:,1]   #comment_text column
Y = nparray[:, 0]   #target column

In [4]:
print("X shape - ", X.shape)
print("Y shape - ", Y.shape)

X shape -  (1804874,)
Y shape -  (1804874,)


In [5]:
X

array(["This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
       "Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",
       'This is such an urgent design problem; kudos to you for taking it on. Very impressive!',
       ..., 'thank you ,,,right or wrong,,, i am following your advice',
       'Anyone who is quoted as having the following exchange, even if apocryphal, would have received my vote! \n\nBessie Braddock: "Winston, you are drunk, and what’s more you are disgustingly drunk."\nWinston Churchill: "Bessie, my dear, you are ugly, and what’s more, you are disgustingly ugly. But tomorrow I shall be sober and you will still be disgustingly ugly."',
       'Students defined as EBD are legally just as disabled and eligible for special services as a developmentally disabled or physically disabled student. \n\nEMOTIONAL AND BEHAVIORAL DISORDER (EBD).\nDefinition. \nAn 

In [6]:
Y

array([0.0, 0.0, 0.0, ..., 0.0, 0.6212121212121212, 0.0], dtype=object)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.01)

In [8]:
print("Train data:")
print("X_train shape - ", X_train.shape)
print("Y_train shape - ", Y_train.shape)
print("Test data")
print("X_test shape - ", X_test.shape)
print("Y_test shape - ", Y_test.shape)

Train data:
X_train shape -  (1786825,)
Y_train shape -  (1786825,)
Test data
X_test shape -  (18049,)
Y_test shape -  (18049,)


In [9]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words, words_to_index, index_to_words, word_to_vec_map

In [10]:
vocabulary, word_to_index, index_to_word, word_to_vec_map = read_glove_vecs("data/glove.6B.50d.txt")

In [11]:
word = "cucumber"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


In [12]:
print("length of word_to_index - ", len(word_to_index))
print("length of word_to_vec_map - ", len(word_to_vec_map))

length of word_to_index -  400000
length of word_to_vec_map -  400000


In [13]:
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [14]:
def sentence_to_avg(X, word_to_vec_map, vocabulary):
    
    m = X.shape[0]                                   # number of training examples
    #print("m - ", m)
    emb_dim = word_to_vec_map["cucumber"].shape[0]    #dimension = 50
    
    X_averages = np.zeros((m,emb_dim))
    
    for i in range(m):
        # Step 1: Split sentence into list of lower case words (≈ 1 line)
        sentence = strip_punctuation(X[i])
        #print("sentence - ", sentence)
        words = sentence.lower().split()
        filtered_words = [word for word in words if word in vocabulary]
        #print("filtered_words - ", filtered_words)
        # Initialize the average word vector, should have the same shape as your word vectors.
        
        vec_sum = np.zeros(emb_dim)
    
        # Step 2: average the word vectors. You can loop over the words in the list "words".
        for w in filtered_words:
            vec_sum += word_to_vec_map[w]
        
        if (len(filtered_words) == 0):
            print(words)
        else:
            avg = vec_sum/len(filtered_words)
            X_averages[i] = avg
        
        if (m % 10000 == 0):
            print("Processed Entries - ", m)
            
    return X_averages

In [15]:
X_averages = sentence_to_avg(X_train, word_to_vec_map, vocabulary)    # this is of shape (1804874, 50)

['bldnm']
['httpswwwadncomopinions20170810thosealaskajunketscanmakeadifference']
['morefakefactsfromdrdeb']
['httpwwwpolicynotecahowhavetaxeschangedoverthelasthalfcentury']
['wwwbusinessinsidercomitturnsoutthatsmokingmarijuanamayactuallymakeyouabetterdriver201112']
['httpswwwconservativereviewcomcommentary201703nosovereigntyallowedroguejudgesgivelegalstandingtofuturehypotheticalimmigrants', 'httpswwwconservativereviewcomcommentary201703theradicaltiesoftheimambehindthetrumpimmigrationlawsuit']
['amorisdeemphasizedthewholeconceptofdoctrinethatscaresburke']
[]
['httpzapatopinetafdb']
['httpshubpagescomliteraturehuntersbeforetenhuntersbeforemen']
['httpstwittercomdalejrstatus912284220067893248']
['ifyouwanttoreadmoreaboutmagicisminthechurchjustlookatthelasttwopostsbytradcath']
['frscaliadid']
['httpswwwyoutubecomwatchvkd3jrcl5ao']
['httpswwwyoutubecomwatchvpht0jqfia']
['httpswwwtheglobeandmailcomnewsnationalontariotoraiseminimumwageto15anhourby2019article35152061']
['httpwwwvirtueonlineorg

['riiiiiiiight']
['httpwwwbreitbartcombigjournalism20150926washingtonpostconfirmshillaryclintonstartedthebirthermovement']
['lugenpresse']
['weallstandtherethepoorespeciallyso']
['httpstownhallcomtipsheetjennifervanlaar20170429casheriffhitsbackatsanctuarystaterhetoricwithstatisticsn2320200']
['httpswikileaksorgclintonemailsemailid32511']
['notreallyinoticedthisinthe1990severythingfromabortiontoporn']
['httpwwwnhcnoaagovclimocp100']
[]
['itwastraditionnotdogma']
['afterforciblyconvertingthejews']
['borderslanguageculture']
['x138000']
['kabaaaam']
['ikaka']
['infenestration']
['agayreligiousmarriagewouldhaveachancemybrotherandhishusbandhadaromancollarofficienttheyarenowdivorcignbutweretogetheratotalof20yearswhichislongerthanmymarriagelasted']
['progressiveslovefrancisitisthetradbishopslikechaputandburkewhorebel']
['httpwwwdoctornerdlovecomwpcontentuploads201603youkeepusingthatworde1458094038124jpg']
['haaaaaaaaaaa']
['httpsenwikipediaorgwikipeopleofthebook']
['wwwmamareevacoza']
['https

['blms']
['httpswwwyoutubecomwatchvjudtcxiqqm0featureyoutube', 'httpswwwyoutubecomwatchvntq1tlbwqifeatureyoutube']
['httpswwwjusticegovpardonobamapardons']
['potkettleblack']
['alotofusnoticedtheravagesstartingin1978']
['httpswwwfacebookcomlonevet']
['ohoh']
['httpwwwentertainmentandsportstodaycomtheemmycelebrityconnectedluxurygiftingsuite']
['caiside']
['mmmmmm']
[]
['doughnuturds']
['httpswwwyoutubecomwatchvkferje5olr0']
[]
['httpswwwyoutubecomwatchvxv0jljgaynq']
['1diot']
['bwahahahahahaha']
['httpswwwmerriamwebstercomdictionaryad20hominem']
['httpwwwtheglobeandmailcomglobeinvestorinsidethemarketboostearningsmanipulateaccountinginvestingarticle32021010']
['welldone']
['raaayyyydaaaahhhhsss']
['httpwwwpolitifactcomtruthometerstatements2015oct26bencarsonfactcheckingbencarsonnaziguns']
['httpwwwontariopccomrdocs05312350hydronewsconferencejune115pdf']
['fakenews']
['qqqqd']
['httpsenwikipediaorgwikiacousticharassmentdevice']
['httpsphysorgnews201012scientistsevidenceuniverseshtml']
['bu

['httphumanserviceshawaiigovblogussupremecourtdecisionallowsreductionofhealthcarecoveragetononcitizensstatewillcontinuecertainbenefits']
['l\xadi\xadk\xade', '\xade\xadl\xade\xada\xadn\xador', '\xadi\xadm\xadp\xadl\xadi\xade\xadd', '\xadi', '\xada\xadm', '\xadi\xadns\xadp\xadir\xade\xadd', 't\xadh\xadat', '\xada\xadny\xadb\xado\xaddy', '\xadc\xada\xadn', '\xadg\xadet', '\xadp\xada\xadi\xadd', '9852', '\xadi\xadn', '\xado\xadn\xade', '\xadm\xado\xadnt\xadh', '\xado\xadn', 't\xadh\xade', '\xadi\xadnt\xader\xadn\xadet', '\xadh\xadav\xade', 'y\xadou', 'r\xade\xada\xadd', 't\xadh\xadis', 'w\xade\xadb', 's\xadit\xade', '\xadg\xado', 't\xado', 't\xadh\xadis', 's\xadit\xade', '\xadh\xado\xadm\xade', 't\xada\xadb', '\xadf\xador', '\xadm\xador\xade', '\xadd\xadet\xada\xadi\xadlw\xad\xad\xadw\xad\xad\xadw\xad\xady\xad\xad\xade\xad\xad\xadl\xad\xad\xadp\xad\xad\xad1\xad\xad\xad\xad\xad\xadc\xad\xad\xado\xad\xad\xadmᴵᴵᴵᴵᴵ']
['maybeinthemidwestintheeasttheybecomenones']
['httpwwwnewsweekcomnationsga

['phansi']
['perfectloveforeachotherisharderthanavoidingsin']
['notreallyitpointsoutunequaljusticeiliketoaddmorethantothephraseblacklivesmattermorethanrespectforpolicemorethanpropertymorethanpolicefearingblacks']
['thengowiththebiblicalpenaltyforleadingachildastray']
['httpswwwyoutubecomwatchvd7trcfsumty']
['httpsyoutubehdzrbl7pmw']
['whoputthepleasurecentersinthegenitalia']
['bigly']
['woolshirt']
['ikikii', 'quyanaqvaa']
['predatesmetroandoprahfollowthelinksbelowandfindrealafaithratherthanfear']
['httpsclimatenasagovevidence']
['httpswwwsnopescomhillaryclintonuraniumrussiadeal']
['yawnzzzzzzzzzzzzzzzzzzzzzz']
['httpswwwyoutubecomwatchvlbqdhrcmy']
['httpswwwyoutubecomwatchvwgkqifpr9to', 'httpswwwyoutubecomwatchvexgkjpjydxq', 'httpswwwyoutubecomwatchv4lkmweovooit112s']
['httpswwwyoutubecomwatchvcakueyeoz7c', 'httpswwwyoutubecomwatchvhmtrdzmdhqut1s', 'httpswwwyoutubecomwatchvp2b7cnafyx8']
['alleluja']
['notjokingusuallywhenarroyoiscoveringyouitisagoodsignyouaremakingafoolofyourself']
['

...

['chesterfieldsi', 'hatedthem']
['unlesswearecallingithubris']
['“ronald', 'crump”']
['httpswwwssagovdisabilityssissihtmla00']
['povertyasofficiallydefinedexcludesbenefitsincludethemandyoufindpovertyismuchlowerbecauseofprogramsagainbenedictharshlycondemnsthelossoftheseasdoesfranciswhoisobviouslyshootingattrumpaseventhebishopshaveasoftodayasfarastheahca']
['justinville']
['whichiswhyitwasgoingstronginthegeorgewbushadministrationgrantedservicedoeshavealiberalbiasliketruth']
[]
['tyvm']
['rotflmao']
['gopfundraising']
['wowjust', 'wowmaga']
['termlimitsplease']
['makeamericahatedagain']
['httpsenwikipediaorgwikienergysubsidies']
['httpwwwgreatlandtaxiandtoursservicecomfairbanksairporttaxicabreservationshtml']
['httpwwwupennedugazette0107gaz09html']
['httpnewsnationalpostcomnewscanadacanadiansmorelikelytobeantisemiticthanamericanspollfinds']
['whatevs']
[]
['whyiamacatholicitcouldhavebeenmcculloughchirstianitythefirst3000years']
['andtraslatedbadlybasedonwordsnotideas']
['httpswwwyoutubeco

['loyaltyandfaitharenotthesamethingindeedtheyaredifferentstillitrustthefaithoffrancisburkenotsomuch']
['hahaha']
['httpswwwyoutubecomwatchvirpzkb5mr1u']
['prejudiceisnotdogmaexeclessiaisgroupdynamics']
['httpsenwikipediaorgwikigulfoftonkinincident', 'httpwwwbellmediacaprpressneworiginaldocumentarypearlharbortheaccuseduncoversstartlingnewrevelationspremieringdecember5onhbocanada2']
['hangem']
['ehehuhng']
['disendorse', 'ahahahaahahaahahaahahahaahahaha']
['onlyintersexedchildrengethormones']
['lmbo']
['deflectionsfromfridaysindictment']
['stillsanders']
['httpjokes4uscompeoplejokescomedianjokesjefffoxworthyjokeshtml', 'httpwwwjokesnewscomsearchsouthernaccent', 'httpwwwtelegraphcouknews20170909britishconductorsackedusmusicfestivaljokelabelledracist', 'httpjokescccomfunnybluecollar3ag45wrulesofthesouthernlifestyle']
['murderisatechnicallegalterm']
['matt19wasondivorcedhereferencedmosesifmosessaidit']
['trumpfe']
['unlessthedistrictisbadlygerrymanderedprimaryupsetswillresultindemocraticwin

['httpswwwyoutubecomwatchvblqxqjhhpk']
['truesheshouldhavenoteditwssillegalalreadyandtrumpwssanidiotfornotknowingthat']
['httpsenwikipediaorgwikiwhat27sthematterwithkansas3f']
['httpswwwfacebookcom989theanswerphotosa11260906556512720801110513815774652659210560904972type3', 'httpswwwfacebookcomrestlesspatriotsphotosa19831288360883634844198021666971291881460118627439type3']
['theydontidentifywiththeasexualsinthemodernclergy']
['httpwwwdrugpolicyorgnews201407worldhealthorganizationcallsdrugdecriminalizationandbroaddrugpolicyreforms', 'httpswwwedxorgcoursemanagingaddictionframeworksuccessfuladelaidexaddictionx0']
['httpwwwfreecriticalthinkingorgclimatechange123anthropogenicglobalwarmingtheory']
['grapesakawine']
['irnbru']
['httpwwwrawstorycom201612trumpisaneerilyperfectmatchwiththisfamous14pointguidetofascism']
['athorebnotsinaitheyfoundthealtarand12pillars']
['fatcist']
['theyclingtofamiliarityoverfaith']
['isthereanaturallawreasonnotbasedonsuperstitionorpastmisogyny']
['badpoliceshouldb

['1no', '2done']
['httpswwwwashingtonpostcompoliticswhenonstagewithafellowworldleadertrumpactsthepart201702130c8e2c5ef21411e6a9b0ecee7ce475fcstoryhtmlutmtermb5ea73270f7c']
['haardvark', '…clicks']
['httpswwwthestarcomnewscanada20171020quebecsbill62declareswaronsunglasseshberthtml']
['httpswwwyoutubecomwatchvj6sctwd6ibs']
['derpity', 'derp']
['hepledgedtodoubledownonafghanistan']
['httpswwwfacebookcomamarazzam9']
['httpwwwfederalwaymirrorcomnewsfederalwaygangsstaketerritoryinschools']
['partoftheethicoflifeisenoughtoeatmyentitlementishighmanyhavemuchlessandsufferaccordinglybecauseofconservates']
['hegaveastatementhewasanisiswannabe']
['httpswwwyoutubecomwatchvsl751cddrzi']
['getnancyhome']
['httpsenwikipediaorgwikireductioadabsurdum']
['nobrainer']
['chinp']
['kamasame']
['bubbye']
['canuckistan']
['secondarysourceandprevatii']
['byebye']
['httpwwwmacleanscapoliticsottawathattimercmpinvestigatorsdroppedamidelectionbombshell']
['britishlawerrsonthesideofthechild']
['sluuuurp']
['httpwwwo

['readingbuchanan']
['hahahahaha', 'pingpingdeflectmaga']
['httpwwwnytimescom20161209usobamarussiaelectionhackhtmlr0']
['httpswwwrichardsherman25compagesfoundation']
['httpwwwsecondenlightenmentorgfuture20church20of20reconciliationpdf']
['moderage']
['thorduh']
['httpwwwncregistercomdailynewsdebatingamorislaetitiaalookahead']
['snickersnicker']
['httpsiimgurcomcinax1xpng']
['soreloser']
['tohavethekingdomofgodyouhadtosellallyouhaveandgiveittothepoorhasanythingchanged']
['obamawatsonchin']
['httpdenvercbslocalcom20170524littletongunstoreburglary', 'httpwwwdenverpostcom20170823smashgrabburglarsdrivestolenjeepcabelasstealguns', 'httpappleinsidercomarticles161205burglarsramsuvintopaloaltoapplestoreinsmashandgrabraid']
['httpswwwgooglecomampscraigmedrednews20170622bearignoredsprayamp']
['ifyouunprivilegeromenaturallawismucheasiertodealwith']
['naturallawbasedmoralityisforhumannotgod']
[]
['blackbearsmatter']
['whaaaaat']
['taxtherich']
['legitimateinflation']
['httpwwwalaskapublicorg2015061

In [16]:
print ("X_averages.shape - ", X_averages.shape)

X_averages.shape -  (1786825, 50)


In [17]:
#Y_train = np.transpose(Y_train)

model = Sequential()
model.add(Dense(50, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_averages, Y_train, epochs=10, batch_size=100, shuffle=True)

Epoch 1/10
1786825/1786825 [==============================] - 34s 19us/step - loss: 0.2921 - acc: 0.7006
Epoch 2/10
1786825/1786825 [==============================] - 33s 18us/step - loss: 0.2882 - acc: 0.7004
Epoch 3/10
1786825/1786825 [==============================] - 33s 18us/step - loss: 0.2875 - acc: 0.7004
Epoch 4/10
1786825/1786825 [==============================] - 32s 18us/step - loss: 0.2871 - acc: 0.7004
Epoch 5/10
1786825/1786825 [==============================] - 33s 18us/step - loss: 0.2868 - acc: 0.7004
Epoch 6/10
1786825/1786825 [==============================] - 33s 19us/step - loss: 0.2866 - acc: 0.7004
Epoch 7/10
1786825/1786825 [==============================] - 32s 18us/step - loss: 0.2865 - acc: 0.7004
Epoch 8/10
1786825/1786825 [==============================] - 33s 18us/step - loss: 0.2863 - acc: 0.7004
Epoch 9/10
1786825/1786825 [==============================] - 33s 18us/step - loss: 0.2862 - acc: 0.7004
Epoch 10/10
1786825/1786825 [==========================

In [18]:
train_loss, train_accuracy = model.evaluate(X_averages, Y_train)

1786825/1786825 [==============================] - 36s 20us/step


In [19]:
print("Train Accuracy = " , train_accuracy)
print("Train Loss = " , train_loss)

Train Accuracy =  0.7004345696977481
Train Loss =  0.28591722137118447


In [21]:
X_test_averages = sentence_to_avg(X_test, word_to_vec_map, vocabulary)
print ("X_test_averages.shape - ", X_test_averages.shape)

['httpsphotographyisnotacrimecom20160708portlandpolicearrestmanpulledextendedcliphandgunprotesters']
['theyalwaystryiftheyhaveamajority']
[]
['httpwwwfreedomworksorgcontentpeterdefaziohassomeexplainingdo']
['httpwwwthegwpfcommethanehydratemayignitenewenergywarinasia']
['easynoyouaredescribingafascist']
['httpsenwikipediaorgwikiolaf']
['racsist']
['httpswwwbrennancenterorgissuescrimeratesamerica']
['httpwwwdoekywpcontentuploads201404sunscreencontaminationofcoralreefspdf']
['modesly']
['antifa']
['𝒩ℯ𝒾𝓁', '𝒶', '𝓉𝓇𝓊ℯ', '𝒾𝓃𝓈𝓅𝒾𝓇𝒶𝓉𝒾ℴ𝓃', '𝒻ℴ𝓇', '𝓉𝒽ℴ𝓈ℯ', '𝓉𝒽𝒶𝓉', '𝓀𝓃ℴ𝓌', '𝒽ℴ𝒸𝓀ℯ𝓎']
['httpwwwactivistpostcom201610pentagoncaughtpayingprfirm540millionmakefaketerroristvideoshtml']
['doddfrank']
['snorrrrrrrrrrre']
['httpswwwpressprogresscacanadarichest1gotalotrichersincethe1980sthemiddleclasshasnot']
['httpwwwsnopescomnytimeswiretaparticles']
['cajones']
['httpswwwfacebookcomdavidgrisham58', 'httpswwwfacebookcompglastfrontierevangelismaboutrefpageinternal']
['httpnsidcorgarcticseaicenewsfiles201509mon

In [22]:
test_loss, test_accuracy = model.evaluate(X_test_averages, Y_test)
print("Test Accuracy = " , test_accuracy)
print("Test Loss = " , test_loss)

18049/18049 [==============================] - 0s 24us/step
Test Accuracy =  0.7002049975067871
Test Loss =  0.2853952414164273


In [23]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_confusion_matrix(y_actu, y_pred, title='Confusion matrix', cmap=plt.cm.gray_r):
    
    df_confusion = pd.crosstab(y_actu, y_pred.reshape(y_pred.shape[0],), rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    df_conf_norm = df_confusion / df_confusion.sum(axis=1)
    
    plt.matshow(df_confusion, cmap=cmap) # imshow
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
    plt.ylabel(df_confusion.index.name)
    plt.xlabel(df_confusion.columns.name)

In [24]:
Y_test_pred = model.predict(X_test_averages)

In [27]:
import csv

with open('pred_output_0423.csv', mode='w') as pred_output:
    output_writer = csv.writer(pred_output, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    for i in range(Y_test.shape[0]):
        output_writer.writerow([X_test[i].encode("utf-8"), Y_test[i], Y_test_pred[i]])

In [101]:
test_df = pandas.read_csv("data/test.csv", index_col="id")
test_df.head()

,comment_text
id,
7000000,Jeff Sessions is another one of Trump's Orwell...
7000001,I actually inspected the infrastructure on Gra...
7000002,No it won't . That's just wishful thinking on ...
7000003,Instead of wringing our hands and nibbling the...
7000004,how many of you commenters have garbage piled ...
